# SWAN sensitivity example

In this notebook we will load the base config options from yaml file and define
different SWAN workspaces for different friction parameters to simulate sensitivity testing

In [1]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy
from pathlib import Path
import yaml

import warnings
warnings.filterwarnings('ignore')

## Workspace basepath

In [2]:
workdir = Path("example_sensitivity")
workdir.mkdir(exist_ok=True)

## Instantiate model

Use a fully-defined config from yaml to instantiate ModelRun with the runtime parameters
and the config definition

In [3]:
# Uncoment below to view the contents of the yaml file

# !cat example_sensitivity.yml

In [4]:
from rompy.swan.config import SwanConfigComponents

conf_dict = yaml.load(open("example_sensitivity.yml"), Loader=yaml.Loader)
config = SwanConfigComponents(**conf_dict)
config

SwanConfigComponents(model_type='swanconfig', template='/source/csiro/rompy/rompy/templates/swancomp', checkout='main', cgrid=REGULAR(model_type='regular', spectrum=SPECTRUM(model_type='spectrum', mdc=36, flow=0.04, fhigh=1.0, msc=None, dir1=None, dir2=None), grid=GRIDREGULAR(model_type='gridregular', xp=110.0, yp=-35.2, alp=4.0, xlen=7.5, ylen=12.5, mx=14, my=24, suffix='c')), startup=STARTUP(model_type='startup', project=PROJECT(model_type='project', name='Test declarative', nr='run1', title1='Friction sensitivity testing', title2=None, title3=None), set=SET(model_type='set', level=0.0, nor=None, depmin=0.05, maxmes=None, maxerr=None, grav=None, rho=None, cdcap=None, inrhog=None, hsrerr=None, direction_convention='nautical', pwtail=None, froudmax=None, icewind=None), mode=MODE(model_type='mode', kind='nonstationary', dim='twodimensional'), coordinates=COORDINATES(model_type='coordinates', kind=SPHERICAL(model_type='spherical', projection='ccm'), reapeating=False)), inpgrid=DataInterf

### Examine components

In [5]:
# Project

print(config.startup.project.render())

PROJECT name='Test declarative' nr='run1' title1='Friction sensitivity testing'


In [6]:
# Friction

print(config.physics.friction.render())

FRICTION JONSWAP CONSTANT cfjon=0.038


### Sensitivity config

In [7]:
def set_experiment(config, cfjon):
    """Return a new config object for a given experiment"""
    new_config = deepcopy(config)
    new_config.startup.project.title2 = f"Jonswap friction cfjon={cfjon}"
    new_config.physics.friction.cfjon = cfjon
    return new_config

In [8]:
from rompy.core.time import TimeRange

period = TimeRange(
    start="2023-01-01T00:00:00",
    end="2023-01-02T00:00:00",
    interval="1h"
)

period

TimeRange(start=datetime.datetime(2023, 1, 1, 0, 0), end=datetime.datetime(2023, 1, 2, 0, 0), duration=datetime.timedelta(days=1), interval=datetime.timedelta(seconds=3600), include_end=True)

In [15]:
from rompy.model import ModelRun

runs = []
for cfjon in [0.01, 0.02, 0.03, 0.04, 0.05]:
    run_id = f"cfjon-{cfjon}"
    new_config = set_experiment(config, cfjon=cfjon)

    print(f"\n{new_config.startup.project.render()}")
    print(f"{new_config.physics.friction.render()}")

    runs.append(ModelRun(
        run_id=run_id,
        config=new_config,
        period=period,
        output_dir=str(workdir)),
    )


PROJECT name='Test declarative' nr='run1' title1='Friction sensitivity testing' title2='Jonswap friction cfjon=0.01'
FRICTION JONSWAP CONSTANT cfjon=0.01

PROJECT name='Test declarative' nr='run1' title1='Friction sensitivity testing' title2='Jonswap friction cfjon=0.02'
FRICTION JONSWAP CONSTANT cfjon=0.02

PROJECT name='Test declarative' nr='run1' title1='Friction sensitivity testing' title2='Jonswap friction cfjon=0.03'
FRICTION JONSWAP CONSTANT cfjon=0.03

PROJECT name='Test declarative' nr='run1' title1='Friction sensitivity testing' title2='Jonswap friction cfjon=0.04'
FRICTION JONSWAP CONSTANT cfjon=0.04

PROJECT name='Test declarative' nr='run1' title1='Friction sensitivity testing' title2='Jonswap friction cfjon=0.05'
FRICTION JONSWAP CONSTANT cfjon=0.05


## Run the model

In [17]:
for run in runs:
    run()

INFO:rompy.model:
INFO:rompy.model:-----------------------------------------------------
INFO:rompy.model:Model settings:
INFO:rompy.model:
run_id: cfjon-0.01
period: 
	Start: 2023-01-01 00:00:00
	End: 2023-01-02 00:00:00
	Duration: 1 day, 0:00:00
	Interval: 1:00:00
	Include End: True

output_dir: example_sensitivity
config: <class 'rompy.swan.config.SwanConfigComponents'>

INFO:rompy.model:-----------------------------------------------------
INFO:rompy.model:Generating model input files in example_sensitivity
INFO:rompy.swan.data:	Writing bottom to example_sensitivity/cfjon-0.01/bottom.grd
INFO:rompy.swan.data:	Writing wind to example_sensitivity/cfjon-0.01/wind.grd
INFO:rompy.model:
INFO:rompy.model:Successfully generated project in example_sensitivity
INFO:rompy.model:-----------------------------------------------------
INFO:rompy.model:
INFO:rompy.model:-----------------------------------------------------
INFO:rompy.model:Model settings:
INFO:rompy.model:
run_id: cfjon-0.02
peri

### Check the workspace

In [22]:
sorted(workdir.glob("*"))

[PosixPath('example_sensitivity/cfjon-0.01'),
 PosixPath('example_sensitivity/cfjon-0.02'),
 PosixPath('example_sensitivity/cfjon-0.03'),
 PosixPath('example_sensitivity/cfjon-0.04'),
 PosixPath('example_sensitivity/cfjon-0.05')]

In [24]:
sorted(Path("example_sensitivity/cfjon-0.03").glob("*"))

[PosixPath('example_sensitivity/cfjon-0.03/INPUT'),
 PosixPath('example_sensitivity/cfjon-0.03/bottom.grd'),
 PosixPath('example_sensitivity/cfjon-0.03/wind.grd')]

In [25]:
input = Path("example_sensitivity/cfjon-0.03/INPUT")
print(input.read_text())

! Rompy SwanConfig
! Template: /source/csiro/rompy/rompy/templates/swancomp
! Generated: 2023-10-16 02:11:30.301323 on rafael-XPS by rguedes


! Startup -------------------------------------------------------------------------------------------------------------------------------------------------------------------------

PROJECT name='Test declarative' nr='run1' title1='Friction sensitivity testing' title2='Jonswap friction cfjon=0.03'

SET level=0.0 depmin=0.05 NAUTICAL

MODE NONSTATIONARY TWODIMENSIONAL

COORDINATES SPHERICAL CCM

! Computational Grid --------------------------------------------------------------------------------------------------------------------------------------------------------------


CGRID REGULAR xpc=110.0 ypc=-35.2 alpc=4.0 xlenc=7.5 ylenc=12.5 mxc=14 myc=24 CIRCLE mdc=36 flow=0.04 fhigh=1.0

! Input Grids -------------------------------------------------------------------------------------------------------------------------------------------------------

## Run the model

Todo

## Plot outputs

Todo